# GRO620 - Activité procédurale 2

Dans cette activité procédurale, nous allons poser les bases dufiltrage numérique d'images. Vous reconnaîtrez des éléments du filtrage numérique que vous avez vu en S4.

Pour chaque question impliqant de la programmation, commencez par discuter de la procédure à suivre pour résoudre le problème. Nous validerons l'approche en classe avant de se lancer dans l'implémentation.

In [ ]:
# Préambule

import os
import numpy as np
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/gro620-e21

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive


## 1. Caractéristiques de la lumière

### Q1.1

Dans cette image synthétique : 

![](https://upload.wikimedia.org/wikipedia/commons/c/cd/Specular_highlight.jpg)

(source: [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Specular_highlight.jpg))

**a)** Quelle(s) partie(s) correspondent à l'illumination diffuse et les reflets spéculaires ?

- Illumination diffuse : *smooth shading*, ou partie de lumière reflétée après absorption qui transitionne graduellement entre les différents niveaux d'illumination (ex. : l'ombre sur les sphères);
- Reflets spéculaires : *shiny highlight*, ou partie de lumière qui est complètement reflétée (points blancs).

**b)** Quelle information est nécessaire pour déterminer les caractéristiques et emplacements exacts des sources de lumières dans cette image ? Répondez en utilisant des éléments de la *Bidirectional Reflectance Distribution Function* (BRDF).

- Angles d'incidence ($\theta_i$, $\phi_i$) 
- Angles de réflexion ($\theta_r$, $\phi_r$)
- Longueur d'onde ($\lambda$)

ou

- Angle d'incidence ($\theta_i$) pour corriger le *foreshortening*
- Direction d'incidence ($\hat{v}_i$)
- Direction de réflexion ($\hat{v}_r$)
- Normale du plan de réflexion ($\hat{n}$)
- Longueur d'onde ($\lambda$)

## 2. Encodage de l'image

Pour les questions suivantes, vous aurez probablement besoin de lire la documentation de cv2.imread et matplotlib.pyplot.imshow :

[imread](https://pythonexamples.org/python-opencv-read-image-cv2-imread/)

[imshow](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.imshow.html)

Le code suivant charge une image et l'affiche en ligne :

In [ ]:
img_BGR = cv2.imread("images_doc/proc1-q3-color.jpeg")
plt.imshow(img_BGR)

(source de l'image: [PixaBay, Pexels](https://www.pexels.com/photo/apartment-architecture-block-blue-534124/))

### Q2.1

**a)** Ouvrez directement l'image dans un autre logiciel (le fichier se trouve dans images_doc/proc1-q3-color.jpeg) et comparez le résultat. Que remarquez vous ?

La photo est encodée en RGB, alors que la librairie CV2 enregistre les images au format BGR.

**b)** Affichez seulement le premier canal de couleurs de l'image. Pensez à analyser la composition de la matrice image que OpenCV vous retourne. Expliquez ensuite ce que vous voyez.

In [ ]:
simfig, plots = plt.subplots(2,2, figsize = (12,8))

plots[0][0].imshow(img_BGR)
plots[0][1].imshow(img_BGR[:, :, 0], cmap="gray") # B
plots[1][0].imshow(img_BGR[:, :, 1], cmap="gray") # G
plots[1][1].imshow(img_BGR[:, :, 2], cmap="gray") # R

**c)** Transformez maintenant l'image pour que les couleurs correspondent à ce que vous voyez en dehors de Jupyter.

In [ ]:
# First method
img_RGB = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2RGB)

# Second method
b, g, r = cv2.split(img_BGR)
img_RGB = cv2.merge([r,g,b])

# Third method
img_RGB = img_BGR.copy()
img_RGB[:,:,0] = img_BGR[:,:,2]
img_RGB[:,:,2] = img_BGR[:,:,0]

plt.imshow(img_RGB)

### Q2.2

Soit cette couleur dans l'espace Y'CbCr (on suppose chaque valeur comme étant encodée sur 8 bits) :

$c = [100, 150, 150]$

Trouvez sa valeur équivalente dans l'espace RGB.

In [ ]:
c = np.array([100,150,150])

img_YCC = np.zeros((1,1,3), dtype=np.uint8)
img_YCC[:] = c

img_RGB = cv2.cvtColor(img_YCC, cv2.COLOR_YCrCb2RGB)
print(img_RGB[0,0,:])

## 3. Filtrage point à point

### Q3.1

Soit cette image (chargée par OpenCV et affichée par matplotlib): 

In [ ]:
img_q31_org = cv2.imread("images_doc/proc2-q1-dock.jpeg")
img_q31_rgb = cv2.cvtColor(img_q31_org, cv2.COLOR_BGR2RGB) # Équivalent de la question Q3.1.c de l'activité procédurale 1.
plt.imshow(img_q31_rgb)

(Source de l'image originale : [Vlada Karpovich, Pexels](https://www.pexels.com/photo/snow-wood-landscape-mountains-4450090/))

Cette fonction affiche l'histogramme des trois composantes de l'image :

In [ ]:
channels = ('r','g','b')
for i, col in enumerate(channels):
    hist = cv2.calcHist([img_q31_rgb], [i], None, [256], [0,256])
    plt.plot(hist,color = col)
    plt.xlim([0,256])

Ajustez la plage dynamique en luminosité de l'image pour qu'elle couvre l'ensemble des valeurs possibles.

In [ ]:
# NOTE: On convertit d'abord en float32 dans la plage [0,1] pour
# simplifier la manipulation des images avec des facteurs non-entiers.
# Matplotlib détecte ceci et affichera l'image correctement.
img_q31_f = np.float32(img_q31_rgb) / 255.0

# Preserves RGB proportions
img_q31_out = img_q31_f.copy()
img_q31_out /= img_q31_f.max()

# Maximizes color channels independently
img_q31_max_rgb = img_q31_f.copy()
img_q31_max_rgb[:,:,0] /= img_q31_f[:,:,0].max()
img_q31_max_rgb[:,:,1] /= img_q31_f[:,:,1].max()
img_q31_max_rgb[:,:,2] /= img_q31_f[:,:,2].max()

simfig, plots = plt.subplots(1,2, figsize = (10,10))
plots[0].imshow(img_q31_out)
plots[1].imshow(img_q31_max_rgb)

channels = ('r','g','b')
simfig, plots = plt.subplots(2,1, figsize = (12,6))
for i, col in enumerate(channels):
    hist = cv2.calcHist([img_q31_out], [i], None, [256], [0,1])
    plots[0].plot(hist,color = col)
    plt.xlim([0,256])

for i, col in enumerate(channels):
    hist = cv2.calcHist([img_q31_max_rgb], [i], None, [256], [0,1])
    plots[1].plot(hist,color = col)
    plt.xlim([0,256])

## Q3.2

Soit maintenant cette image :

In [ ]:
img_q32_org = cv2.imread("images_doc/proc2-q1-object.jpeg")
img_q32_rgb = cv2.cvtColor(img_q32_org, cv2.COLOR_BGR2RGB) # Équivalent de la question Q3.1.c de l'activité procédurale 1.
plt.imshow(img_q32_rgb)

Tentez de mettre en place un algorithme basé sur la luminosité permettant d'éliminer l'arrière-plan de cette image pour qu'il ne reste que l'objet en jaune sur un fond le plus noir possible.

In [ ]:
img_q32_filt = img_q32_rgb.copy()
W = img_q32_filt.shape[1] # NOTE: L'ordre des dimensions est Y puis X ("row-major")
H = img_q32_filt.shape[0]

threshold = 180

# First method - max of any channel
img_01 = img_q32_rgb.copy()
for i in range(W):
    for j in range(H):
        if img_01[i,j].max() < threshold:
            img_01[i,j] = np.zeros(3)
            
# Second method - mean of channels
img_02 = img_q32_rgb.copy()
for i in range(W):
    for j in range(H):
        if img_02[i,j].mean() < threshold:
            img_02[i,j] = np.zeros(3)
            
# Third method - gray colormap
img_03 = img_q32_rgb.copy()
img_03 = cv2.cvtColor(img_03, cv2.COLOR_RGB2GRAY)
_, img_03 = cv2.threshold(img_03, threshold, 255, cv2.THRESH_BINARY)
            
# Fourth method - Otsu method
img_04 = img_q32_rgb.copy()
img_04 = cv2.cvtColor(img_04, cv2.COLOR_BGR2GRAY)
_, img_04 = cv2.threshold(img_04, threshold, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
img_04 = cv2.findContours(img_04, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2]
img_04 = sorted(img_04, key=cv2.contourArea)
for i in img_04:
    if cv2.contourArea(i) > 250: break

mask = np.zeros(img_q32_rgb.shape[:2], np.uint8)
cv2.drawContours(mask, [i],-1, 255, -1)
img_04 = cv2.bitwise_and(img_q32_rgb, img_q32_rgb, mask=mask)

# Fifth method - HSV
img_05 = img_q32_rgb.copy()
img_05 = cv2.cvtColor(img_05, cv2.COLOR_RGB2HSV)
v_value = img_05[100,100,2]
lower = np.array([0,0,v_value], np.uint8)
upper = np.array([threshold, 255, 255], np.uint8)
mask = cv2.inRange(img_05, lower, upper)
img_05 = cv2.bitwise_and(img_q32_rgb, img_q32_rgb, mask=mask)

if not os.path.exists('./output'):
    os.makedirs('./output')
    
simfig, plots = plt.subplots(1,5, figsize = (16,10))
plots[0].imshow(img_01)
plots[1].imshow(img_02)
plots[2].imshow(img_03, cmap="gray")
plots[3].imshow(img_04)
plots[4].imshow(img_05)
simfig.savefig('output/q32.pdf', dpi=300)

cv2.imwrite('output/q32_max.jpg', cv2.cvtColor(img_01, cv2.COLOR_RGB2BGR))
cv2.imwrite('output/q32_mean.jpg', cv2.cvtColor(img_02, cv2.COLOR_RGB2BGR))
cv2.imwrite('output/q32_gray.jpg', cv2.cvtColor(img_03, cv2.COLOR_RGB2BGR))
cv2.imwrite('output/q32_otsu.jpg', cv2.cvtColor(img_04, cv2.COLOR_RGB2BGR))
cv2.imwrite('output/q32_hsv.jpg', cv2.cvtColor(img_05, cv2.COLOR_RGB2BGR))

## 4. Filtrage linéaire

### Q4.1 

Soit l'image suivante ainsi que sa transformée de Fourier :

In [ ]:
img_q4_org  = cv2.imread("images_doc/proc2-q2-texture.jpeg")
img_q4_mono = np.float32(cv2.cvtColor(img_q4_org, cv2.COLOR_BGR2GRAY)) / 255.0

def get_fft_mag(img):
    img_fft = np.fft.fft2(img)
    img_fft = np.fft.fftshift(img_fft)
    img_fft = 20*np.log(np.abs(img_fft))
    return img_fft
    
img_q4_mono_fft = get_fft_mag(img_q4_mono)

plt.subplot(121),plt.imshow(img_q4_mono, cmap="gray")
plt.subplot(122),plt.imshow(img_q4_mono_fft)

(Source de l'image originale : [Hoang Le, Pexels](https://www.pexels.com/photo/black-and-white-black-and-white-pattern-rough-978462/)).

**a)** Filtrez cette image à l'aide d'une convolution de façon à ce que la valeur de chaque pixel soit la valeur moyenne de ses voisins dans un carré de 15x15.

In [ ]:
dim = 15
filter_kernel = np.ones((dim,dim),np.float32)/dim**2
img_q4_conv = cv2.filter2D(img_q4_mono,-1,filter_kernel)
img_q4_conv_fft = get_fft_mag(img_q4_conv)

simfig, plots = plt.subplots(1,2, figsize = (10,6))
plots[0].imshow(img_q4_mono, cmap="gray")
plots[1].imshow(img_q4_mono_fft)
simfig, plots = plt.subplots(1,2, figsize = (10,6))
plots[0].imshow(img_q4_conv, cmap="gray")
plots[1].imshow(img_q4_conv_fft)

**b)** Comparez le résultat avec celui de la fonction cv2.GaussianBlur() avec un noyau de convolution de la même taille.

In [ ]:
img_q4_blur = cv2.GaussianBlur(img_q4_mono, (dim,dim), 0)
img_q4_blur_fft = get_fft_mag(img_q4_blur)

simfig, plots = plt.subplots(1,2, figsize = (10,6))
plots[0].imshow(img_q4_mono, cmap="gray")
plots[1].imshow(img_q4_mono_fft)
simfig, plots = plt.subplots(1,2, figsize = (10,6))
plots[0].imshow(img_q4_conv, cmap="gray")
plots[1].imshow(img_q4_conv_fft)
simfig, plots = plt.subplots(1,2, figsize = (10,6))
plots[0].imshow(img_q4_blur, cmap="gray")
plots[1].imshow(img_q4_blur_fft)

**c)** Comment expliquez-vous la différence ?

La moyenne mobile accentue certains pixels au lieu de les rendre diffus, ce qui crée de l'*aliasing* à l'intérieur de l'image. Ce n'est pas le cas avec la gaussienne, qui elle permet d'éviter l'accentuation grâce à sa distribution.

### Q4.2

Utilisez un filtre linéaire pour extraire les contours de l'image fournie en Q3.2.

In [ ]:
# First method
img_q32_contour = img_q32_rgb.copy()
filter_kernel = np.array([[1,0,-1],[0,0,0],[-1,0,1]])
img_q32_contour = cv2.cvtColor(img_q32_contour, cv2.COLOR_RGB2GRAY)
img_q32_contour = cv2.filter2D(img_q32_contour,-1,filter_kernel)

# Second method
img_q32_contour = img_q32_rgb.copy()
dim = 15
minVal = 150
maxVal = 200
img_q32_contour = cv2.GaussianBlur(img_q32_contour, (15,15), 0)
img_q32_contour = cv2.Canny(img_q32_contour, minVal, maxVal)

# Third method
# ./input/q42_Sobel.png

# Fourth method
# ./input/q42_filter2D.png

plt.imshow(img_q32_contour, cmap="gray")

## 5. Filtrage morphologique et chaîne de traitement

### Q5.1 

À partir de l'image de la question Q1.1, combinez les filtres vus plus tôt pour ne conserver que le contour de l'objet de la figure (donc sans bruit de fond).

In [ ]:
img_BGR = cv2.imread("images_doc/proc1-q3-color.jpeg")
img_RGB = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2RGB)

dim = 15
minVal = 0
maxVal = 150
img_RGB = cv2.GaussianBlur(img_RGB, (15,15), 0)
img_RGB = cv2.Canny(img_RGB, minVal, maxVal)

plt.imshow(img_RGB, cmap="gray")